In [ ]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 4.7 MB/s 
     |████████████████████████████████| 16.6 MB 41.9 MB/s 
     |████████████████████████████████| 7.8 MB 67.0 MB/s 
     |████████████████████████████████| 1.6 MB 63.0 MB/s 
     |████████████████████████████████| 128 kB 74.8 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 132 kB 57.8 MB/s 


In [1]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
else:
    !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'RI',
    root_path = root_path,
)

refresh = [
    # 'shapes',
    # 'pl',
    # 'assignments',
    # 'plans',
    # 'geo',
    # 'crosswalks',
    # 'transformers',
    # 'acs5',
]

for ds in self.bq.client.list_datasets():
    if ds.dataset_id in refresh:
        self.bq.del_ds(ds)

level='vtd'
year = 2018
# self.get_shapes(overwrite=True)
# self.get_pl(overwrite=True)
# self.get_assignments(overwrite=True)
# self.get_plans(overwrite=True)
# self.get_geo(overwrite=True)
# self.get_crosswalks(overwrite=True)
# self.get_transformer(overwrite=True)
# self.get_acs5(rear=year, overwrite=True)
self.transform_acs5(level=level, year=year, extra_cols=('geometry'))#, overwrite=True
self.get_elections()#overwrite=True)
# self.get_geo(level='vtd')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
creating acs5.RI_tract2018 ... Can not compute other_vap_spanishAtHome ... Can not compute other_vap_spanishAtHomeEnglishWell ... Elapsed time: 37.1872 seconds
creating crosswalks.RI_block2020_raw ... downloading from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST44.zip ... done ... Elapsed time: 6.3187 seconds
creating shapes.RI_block2020 ... downloading from https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/tl_2020_44_tabblock20.zip ... done ... Elapsed time: 44.0640 seconds
creating pl.RI_block2020 ... Elapsed time: 14.2664 seconds
creating assignments.RI_block2020 ... downloading from https://www2.census.gov/geo/docs/maps-data/data/baf2020/BlockAssign_ST44_RI.zip ... done ... Elapsed time: 5.4369 seconds
creating geo.RI_block2020 ... Elapsed time: 16.0661 seconds
creating assignments.RI_block2010 ... Elapsed time:

'acs5.RI_tract2018_vtd2020'

In [6]:
from codetiming import Timer as Timer
with Timer("some_name"):
    for i in range(100):
        1+1
T = Timer()
T([x for x in range(10)])

Elapsed time: 0.0000 seconds


<function contextlib.ContextDecorator.__call__.<locals>.inner>

In [5]:
select * except (geometry), geometry,
from (
    select
        *,
        st_perimeter(geometry) as perim,
        case when perim < 0.1 then 0 else 4 * 3.141592653589793 * atot / (perim * perim) end as polsby_popper,
    from (
        select
            vtd2020,
            min(dist_to_border) as dist_to_border,
            sum(aland) as aland,
            sum(awater) as awater,
            sum(atot) as atot,
            st_union_agg(geometry) as geometry,
        from geo.RI_block2020
        group by vtd2020
    ) as S
join (
    select vtd2020, sum(all_tot_pop) as all_tot_pop, sum(all_vap_pop) as all_vap_pop, sum(white_tot_pop) as white_tot_pop, sum(white_vap_pop) as white_vap_pop, sum(hisp_tot_pop) as hisp_tot_pop, sum(hisp_vap_pop) as hisp_vap_pop, sum(other_tot_pop) as other_tot_pop, sum(other_vap_pop) as other_vap_pop,
    from geo.RI_block2020
    group by vtd2020
) as P
using (vtd2020)
join (
    select vtd2020, county2020, congress2010, senate2010, house2010
    from geo.RI_block2020
    qualify row_number() over (partition by vtd2020 order by all_tot_pop desc) = 1
) as D
using (vtd2020)

'02'

In [ ]:
from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp
geoid='block2020'
path = pathlib.Path('/content/drive/MyDrive/gerrymandering/2022-12/data/plans/')
# pd.read_csv(f, names=['block2020', 'hi'], header=0)
L = [ut.prep(pd.read_csv(file, names=[geoid, file.stem], header=0)).set_index(geoid) for file in path.iterdir() if file.suffix == '.csv']

In [ ]:
df = pd.concat(L, axis=1)

In [ ]:
df.nunique(axis=0)

plans2100    31
plans2101    31
plans2103     3
plans2108    14
plans2109    31
             ..
planc2185     3
planc2186    38
planc2193    38
planc2194    38
planc2195    17
Length: 242, dtype: int64

In [ ]:
T = [
    # 'redistricting-361203.shapes.RI_block2020',
    # 'redistricting-361203.pl.RI_block2020',
    'redistricting-361203.assignments.RI_block2020',
    'redistricting-361203.geo.RI_block2020',
    # 'redistricting-361203.crosswalks.RI_block2020_raw',
    'redistricting-361203.crosswalks.RI_block2020',
    'redistricting-361203.transformers.RI_tract2018',
]
x = 'vtd2020'
for tbl in T:
    
    qry = f'select count(*) from (select distinct {x} FROM {tbl})'
    print(qry)
    print(self.bq.qry_to_df(qry).squeeze())
    

select count(*) from (select distinct vtd2020 FROM redistricting-361203.assignments.RI_block2020)
423
select count(*) from (select distinct vtd2020 FROM redistricting-361203.geo.RI_block2020)
423
select count(*) from (select distinct vtd2020 FROM redistricting-361203.crosswalks.RI_block2020)
423
select count(*) from (select distinct vtd2020 FROM redistricting-361203.transformers.RI_tract2018)
423


In [ ]:
vp.unzipper('/home/content/sample_data/mnist_test.csv')

In [ ]:
refresh = [
    # 'shapes',
    # 'pl',
    # 'assignments',
    'geo',
    'crosswalks',
    'transformers',
    'acs5',
]

for ds in self.bq.client.list_datasets():
    # if ds.dataset_id in refresh:
    print(ds.dataset_id, ds.dataset_id in refresh)
        # self.bq.del_ds(ds)


acs5 True
assignments False
crosswalks True
pl False
shapes False
transformers True


In [ ]:
subpops = {
    'all_tot_pop'  : 'p2_001n',
    'all_vap_pop'  : 'p4_001n',
    'wite_tot_pop' : 'p2_005n',
    'white_vap_pop': 'p4_005n',
    'hisp_tot_pop' : 'p2_002n',
    'hisp_vap_pop' : 'p4_002n',
    'other_tot_pop': None,
    'other_vap_pop': None,
}
{v:k for k,v in subpops.items()}
# dict(zip(d.items()))

{'p2_001n': 'all_tot_pop',
 'p4_001n': 'all_vap_pop',
 'p2_005n': 'wite_tot_pop',
 'p4_005n': 'white_vap_pop',
 'p2_002n': 'hisp_tot_pop',
 'p4_002n': 'hisp_vap_pop',
 None: 'other_vap_pop'}

In [ ]:
x = 'ababab'
x[:x.rfind('a')]

'abab'